# Experiments with the environment classes

In [2]:
# allow imports from the main source directory
import sys
sys.path.append('..')

import timeit
import pathlib
import shutil
import numpy as np
import matplotlib.pyplot as plt
from Environment import EpidemicSpreadEnvironment, DissipationModelEnvironment, PrecalculatedEnvironment, animate_environment


from IPython.display import display, HTML
import logging
logging.basicConfig(level=logging.WARNING)
# %matplotlib inline
%matplotlib widget

# Environment base classes

The basic abstract classes that model certain kind of environments and their temporal evolution and the way they can be observed.

* Tracks an observable of the environment (eg. temperature). 
* Tracks the available information about the environment. 

## Visualizing the environment

In [5]:

env = EpidemicSpreadEnvironment("crop", 100, 100, seed=40, infection_duration = 5, p_transmission = 0.1)
env.status[6,10] = 2
env.status[60,80] = 5
# make a filtered area which won't be susceptible 
env.status[10:50, 10:50] = -2
anim = animate_environment(env)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Testing the speed
Trying out different sizes of EpidemicSpreadEnvironment and DissipationModelEnvironment, and testing how long the creation and the proceed operation takes on them

In [3]:
for i in [10, 50, 100, 200, 400, 1000, 2000, 4000]:
    env = EpidemicSpreadEnvironment("crop", i, i, seed=40, infection_duration = 5, p_transmission = 0.1)
    env.status[i // 2, i // 2] = 2
    env.status[(3*i) // 4, (3*i) // 4] = 5
    time = timeit.timeit("env.proceed(1.0)", number=1, globals=globals())
    print(f"map of size {i}x{i} a proceed took {time:0.2} seconds")
# trying out different sizes of DissipationModelEnvironment
print("DissipationModelEnvironment")
for i in [10, 50, 100, 200, 400, 1000, 2000, 4000]:
    env = DissipationModelEnvironment("pollution", i, i, seed=40)
    # env.status[i // 2, i // 2] = 2
    # env.status[(3*i) // 4, (3*i) // 4] = 5
    time = timeit.timeit("env.proceed(1.0)", number=1, globals=globals())
    print(f"map of size {i}x{i} a proceed took {time:0.2} seconds")

map of size 10x10 a proceed took 0.00014 seconds
map of size 50x50 a proceed took 0.0012 seconds
map of size 100x100 a proceed took 0.0024 seconds
map of size 200x200 a proceed took 0.0096 seconds
map of size 400x400 a proceed took 0.038 seconds
map of size 1000x1000 a proceed took 0.24 seconds
map of size 2000x2000 a proceed took 0.94 seconds
map of size 4000x4000 a proceed took 3.9 seconds
DissipationModelEnvironment
map of size 10x10 a proceed took 4.5e-05 seconds
map of size 50x50 a proceed took 9.7e-05 seconds
map of size 100x100 a proceed took 0.00054 seconds
map of size 200x200 a proceed took 0.0011 seconds
map of size 400x400 a proceed took 0.0041 seconds
map of size 1000x1000 a proceed took 0.026 seconds
map of size 2000x2000 a proceed took 0.12 seconds
map of size 4000x4000 a proceed took 0.48 seconds


## Testing the saving model

/home/lboloni/Documents/HackingWork/2022-01-15-MREM
/home/lboloni/Documents/HackingWork/2022-01-15-MREM/__Temporary/2022-01-15-MREM_data/experiments


In [5]:
p = pathlib.Path.cwd().parent.parent.parent
#print(p)
savedir = pathlib.Path(p.parent, "__Temporary", p.name + "_data", "experiments")
#print(savedir)
shutil.rmtree(savedir)
savedir.mkdir(parents=True, exist_ok = True)

height = 20 # was 2000
width = 20 # was 2000
env = EpidemicSpreadEnvironment("crop", width, height, seed=40, infection_duration = 5, p_transmission = 0.1)
env.status[width // 2, height // 2] = 2
env.status[(3*width) // 4, (3*height) // 4] = 5
precenv = PrecalculatedEnvironment(width,height, env, savedir)
for t in range(0, 100):
    precenv.proceed(1.0)

## reloading
pe = PrecalculatedEnvironment(width, height, None, savedir)
anim = animate_environment(pe)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Creating a spread area

In [3]:
dimension = 7
center = dimension // 2 + 1
neighborcounting_kernel = np.zeros((dimension, dimension))
for i in range(dimension):
    for j in range(dimension):
        dist2 = (i - center)*(i - center) + (j - center)*(j - center)
        if dist2 != 0.0:
            neighborcounting_kernel[i,j] = 1 / dist2
print(neighborcounting_kernel)


[[0.03125    0.04       0.05       0.05882353 0.0625     0.05882353
  0.05      ]
 [0.04       0.05555556 0.07692308 0.1        0.11111111 0.1
  0.07692308]
 [0.05       0.07692308 0.125      0.2        0.25       0.2
  0.125     ]
 [0.05882353 0.1        0.2        0.5        1.         0.5
  0.2       ]
 [0.0625     0.11111111 0.25       1.         0.         1.
  0.25      ]
 [0.05882353 0.1        0.2        0.5        1.         0.5
  0.2       ]
 [0.05       0.07692308 0.125      0.2        0.25       0.2
  0.125     ]]
